# <a href=https://eprint.iacr.org/2024/585>A Complete Beginner Guide to the Number Theoretic Transform (NTT)</a>

In [1]:
import numpy as np

# Cyclic Convolution

$\textbf{Definition}$

Suppose that $G(x)$ and $H(x)$ are polynomials with degree $n-1$ in the quotient ring $\Z[x]/(x^n-1)$ where $q\in \Z$.

A $\textbf{cyclic convolution}$ or $\textbf{positive wrapped convolution}$, $\textsf{PWC}(x)$ is defined as:

\begin{align*}
    \textsf{PWC}(x) = \sum_{k=0}^{n-1} c_k x^k \text{ where } c_k = \sum_{i=0}^{k} g_i h_{k-1} + \sum_{i=k+1}^{n-1} g_i h_{k+n-i} \mod q.
\end{align*}

Let $Y(x)$ is the result of linear convolution in the ring $\Z[x]$, it also can be defined as:

\begin{align*}
    \textsf{PWC}(x) = Y(x) \mod (x^n-1).
\end{align*}

In [2]:
def PWC(poly1, poly2):
    n = len(poly1)
    result = []
    for k in range(n):
        v = 0
        for i in range(k+1):
            v += poly1[i] * poly2[k-i]
        for i in range(k+1, n):
            v += poly1[i] * poly2[(k + n - i)]

        result.append(v)
    return result

poly1 = [1, 2, 3, 4]
poly2 = [5, 6, 7, 8]
print("Result of PWC : ", PWC(poly1, poly2))

poly1 = np.array([4, 3, 2, 1])  # 1 + 2X + 3X^2 + 4X^3
poly2 = np.array([8, 7, 6, 5])  # 4 + 3X + 2X^2 + 1X^3
divisor = np.array([1, 0, 0, 0, -1])  # X^4 + 1
product = np.polymul(poly1, poly2)
quotient, remainder = np.polydiv(product, divisor)
print("Remainder of Polydiv : ", remainder[::-1])

Result of PWC :  [66, 68, 66, 60]
Remainder of Polydiv :  [66. 68. 66. 60.]


---
# Negacyclic Convolution

> $\textbf{Definition.}$ 
> Suppose that $G(x)$ and $H(x)$ are polynomials with degree $n-1$ in the quotient ring $\mathbb{Z}[x]/(x^n+1)$ where $q \in \mathbb{Z}$.
>
> A $\textbf{negacyclic convolution}$ or $\textbf{negative wrapped convolution}$, $\textsf{NWC}(x)$ is defined as : 
> 
> \begin{align*}
>   \textsf{NWC}(x) = \sum_{k=0}^{n-1}c_kx^k \text{, where } c_k = \sum_{i=0}^{k}g_ih_{k-i} - \sum_{i=k+1}^{n-1}g_ih_{k+n-i} \mod q.
> \end{align*}
>
> Let $Y(x)$ is the result of linear convolution in the ring $\mathbb{Z}[x]$, it also can be defined as
>
> \begin{align*}
>       \textsf{NWC}(x) = Y(x) \mod (x^n+1).
> \end{align*}

In [3]:
def NWC(poly1, poly2):
    n = len(poly1)
    result = []
    for k in range(n):
        v = 0
        for i in range(k+1):
            v += poly1[i] * poly2[k-i]
        for i in range(k+1, n):
            v -= poly1[i] * poly2[(k + n - i)]

        result.append(v)
    return result

poly1 = [1, 2, 3, 4]
poly2 = [5, 6, 7, 8]
print("Result of NWC : ", NWC(poly1, poly2))

poly1 = np.array([4, 3, 2, 1])  # 1 + 2X + 3X^2 + 4X^3
poly2 = np.array([8, 7, 6, 5])  # 4 + 3X + 2X^2 + 1X^3
divisor = np.array([1, 0, 0, 0, 1])  # X^4 + 1
product = np.polymul(poly1, poly2)
quotient, remainder = np.polydiv(product, divisor)
print("Remainder of Polydiv : ", remainder[::-1])

Result of NWC :  [-56, -36, 2, 60]
Remainder of Polydiv :  [-56. -36.   2.  60.]


---
# NTT-based Cyclic Convolution

>$\textbf{Definition}$
>
>Let $\Z_q$ be an integer ring modulo $q$, and $n-1$ is the polynomial degree of $G(x)$ and $H(x)$ and $\omega$ is its primitive $n$-th root of unity.
>
>Such rings have a multiplicative identity (unity) of 1.
>
>Define $\omega$ as the primitive $n$-th root of unity in $\Z_q$ iff
>
>$$
>    \omega^n \equiv \mod q \text{ and } \omega^k \not\equiv 1 \mod q \text{ for } k<n .
>$$

>$\textbf{Definition}$
>
>The Number Theoretic Transform (NTT) of a vector of polynomial coefficients $\bm{a}$ is defined as $\hat{\bm{a}} = \textsf{NTT}(\bm{a})$, where:
>
>$$
>    \hat{\bm{a}}_j = \sum_{i=0}^{n-1} \omega^{ij}a_i \mod q 
>$$
>
>and $j=0,1, \dots, n-1$ .

>$\textbf{Definition}$
>
>The inverse of Number Theoretic Transform ($\textsf{INTT}$) of an NTT vector $\hat{\bm{a}}$ is defined as $\bm{a} = \textsf{INTT}(\hat{\bm{a}})$, where:
>
>$$
>    \bm{a}_i = n^{-1} \sum_{j=0}^{n-1} \omega^{-ij} \hat{a}_j \mod q,
>$$
>
>and $j=0, 1, 2, \dots, n-1.$

Note that when 
* $n=4$
* $\omega = 3383$

then
* $n^{-1} = 5761$
* $\omega^{-1} = 4298$

In [4]:
def positive_wrapped_ntt(a, omega, q):
    n = len(a)
    # Initialize the result vector with zeros
    hat_a = np.zeros(n, dtype=int)
    
    # Loop over each j to compute hat_a[j]
    for j in range(n):
        # Summation: hat_a[j] = sum(omega^(ij) * a_i) mod q
        summation = 0
        for i in range(n):
            exponent = (i * j) % (n) # exponent ij modulo n
            summation += pow(omega, exponent, q) * a[i]
        
        # Take modulo q for the result
        hat_a[j] = summation % q
    
    return hat_a

def positive_wrapped_intt(hat_a, omega_inv, q, n_inv):
    # Initialize the result vector with zeros
    a = np.zeros(n, dtype=int)
    
    # Loop over each j to compute hat_a[j]
    for i in range(n):
        # Summation: a[i] = sum(omega_inv^{-(ij)} * hat_a[j]) mod q
        summation = 0
        for j in range(n):
            exponent = (i * j) % (n) # exponent 2ij + i modulo 2n
            summation += pow(omega_inv, exponent, q) * hat_a[j]
        
        # Take modulo q for the result
        a[i] = (n_inv * summation) % q
    
    return a

# Example usage:

# Polynomial coefficients a = [a_0, a_1, ..., a_{n-1}]
a = [1, 2, 3, 4]  # Example input polynomial
b = [5, 6, 7, 8]  # Example input polynomial
n = 4    # Degree n-1
n_inv = 5761
q = 7681  # Modulo value q
omega = 3383  # A primitive root or some suitable value for psi
omega_inv = 4298

# Compute the Positive-Wrapped NTT
hat_a = positive_wrapped_ntt(a, omega, q)
hat_b = positive_wrapped_ntt(b, omega, q)

a = positive_wrapped_intt(hat_a, omega_inv, q, n_inv)
b = positive_wrapped_intt(hat_b, omega_inv, q, n_inv)


print("NTT  of the polynomial coefficients : ", hat_a)
print("NTT  of the polynomial coefficients : ", hat_b)
print("INTT of the polynomial coefficients : ", a)
print("INTT of the polynomial coefficients : ", b)

NTT  of the polynomial coefficients :  [  10  913 7679 6764]
NTT  of the polynomial coefficients :  [  26  913 7679 6764]
INTT of the polynomial coefficients :  [1 2 3 4]
INTT of the polynomial coefficients :  [5 6 7 8]


>$\textbf{Proposition}$
>
>Let $\bm{a}$ and $\bm{b}$ are the multiplicands' vectors of polynomial coefficients.
>
>The positive-wrapped convolution of $\bm{a}$ and $\bm{b}$ can be calculated by:
>
>\begin{align*}
>    \bm{c} = \textsf{INTT}(\textsf{NTT}(\bm{a}) \circ \textsf{NTT}(\bm{b}))
>\end{align*}
>
>where $\circ$ is an element-wise multiplication in $\Z_q$.

In [5]:
def NTT_like_PWC(poly1, poly2, omega, omega_inv, n, n_inv, q):
    ntt_poly1 = positive_wrapped_ntt(poly1, omega, q)
    ntt_poly2 = positive_wrapped_ntt(poly2, omega, q)

    ntt_res = np.zeros(n)
    for i in range(n):
        ntt_res[i] = (ntt_poly1[i] * ntt_poly2[i]) % q
    
    poly_nega_conv = positive_wrapped_intt(ntt_res, omega_inv, q, n_inv)
    return poly_nega_conv

# Polynomial coefficients a = [a_0, a_1, ..., a_{n-1}]
a = [1, 2, 3, 4]  # Example input polynomial
b = [5, 6, 7, 8]  # Example input polynomial
n = 4    # Degree n-1
n_inv = 5761
q = 7681  # Modulo value q
omega = 3383  # A primitive root or some suitable value for psi
omega_inv = 4298

res = NTT_like_PWC(a, b, omega, omega_inv, n, n_inv, q)
print(res)

[66 68 66 60]


---
# NTT-based Negacyclic Convolution

>$\textbf{Definition}$
>
>Let $\Z_q$ be an integer ring modulo $q$, and $n-1$ is the polynomial degree of $G(x)$ and $H(x)$ and $\omega$ is its primitive $n$-th root of unity.
>
>Define $\psi$ as the primitive $2n$-th root of unity iff
>
>$$
>    \psi^2 \equiv \omega \mod q \text{     and      } \psi^n \equiv -1 \mod q.
>$$




>$\textbf{Definition}$ 
>
>The Negative-Wrapped Number Theoretic Transform($\text{NTT}^\psi$) of a vector of polynomial coefficients $\bm{a}$ is defined as $\hat{\bm{a}}$ = $\text{NTT}^{\psi}(\bm{a})$, where
>
>\begin{align*}
>    \hat{a}_j = \sum_{i=0}^{n-1} \psi^i \omega^{ij} a_i \mod q, 
>\end{align*}
>
>and $j=0,1,2, \dots, n-1$. As $\psi^2 \equiv \omega \mod q$, we can substitute $\omega = \psi^2$,
>
>\begin{align*}
>    \hat{\bm{a}}_j = \sum_{i=0}^{n-1} \psi^{2ij + i}a_i \mod q.
>\end{align*}

>$\textbf{Definition}$
>
>The Negative-Wrapped Inverse of Number Theoretic Transform (INTT) of an NTT vector $\bm{\hat{a}}$ is defined as $\bm{a} = \textsf{INTT}^{\psi^{-1}}(\hat{\bm{a}})$, where
>
>\begin{align*}
>    \bm{a}_i = n^{-1} \sum_{j=0}^{n-1} \psi^{-1} \omega^{-ij} \hat{a}_j \mod q 
>\end{align*}
>
>and $i=0, 1, 2, \dots, n-1.$ Substiituting $\omega = \psi^2$ yields:
>
>\begin{align*}
>    \bm{a}_i = n^{-1} \sum_{j=0}^{n-1} \psi^{-(2ij+i)} \hat{a}_j \mod q
>\end{align*}

Note that when 
* $n=4$
* $\psi = 1925$

then
* $n^{-1} = 5761$
* $\psi^{-1} = 1213$

In [6]:
def negative_wrapped_ntt(a, psi, q):
    n = len(a)
    # Initialize the result vector with zeros
    hat_a = np.zeros(n, dtype=int)
    
    # Loop over each j to compute hat_a[j]
    for j in range(n):
        # Summation: hat_a[j] = sum(psi^(2ij + i) * a_i) mod q
        summation = 0
        for i in range(n):
            exponent = (2 * i * j + i) % (2*n) # exponent 2ij + i modulo 2n
            summation += pow(psi, exponent, q) * a[i]
        
        # Take modulo q for the result
        hat_a[j] = summation % q
    
    return hat_a

def negative_wrapped_intt(hat_a, psi_inv, q, n_inv):
    # Initialize the result vector with zeros
    a = np.zeros(n, dtype=int)
    
    # Loop over each j to compute hat_a[j]
    for i in range(n):
        # Summation: a[i] = sum(psi_inv^-{(2ij + j)} * hat_a[j]) mod q
        summation = 0
        for j in range(n):
            exponent = (2 * i * j + i) % (2*n) # exponent 2ij + i modulo 2n
            summation += pow(psi_inv, exponent, q) * hat_a[j]
        
        # Take modulo q for the result
        a[i] = (n_inv * summation) % q
    
    return a

# Example usage:

# Polynomial coefficients a = [a_0, a_1, ..., a_{n-1}]
a = [1, 2, 3, 4]  # Example input polynomial
b = [5, 6, 7, 8]  # Example input polynomial
n = 4    # Degree n-1
n_inv = 5761
q = 7681  # Modulo value q
psi = 1925  # A primitive root or some suitable value for psi
psi_inv = 1213

# Compute the Negative-Wrapped NTT
hat_a = negative_wrapped_ntt(a, psi, q)
hat_b = negative_wrapped_ntt(b, psi, q)

a = negative_wrapped_intt(hat_a, psi_inv, q, n_inv)
b = negative_wrapped_intt(hat_b, psi_inv, q, n_inv)


print("NTT^ψ of the polynomial coefficients:", hat_a)
print("NTT^ψ of the polynomial coefficients:", hat_b)
print("INTT^ψ of the polynomial coefficients:", a)
print("INTT^ψ of the polynomial coefficients:", b)

NTT^ψ of the polynomial coefficients: [1467 2807 3471 7621]
NTT^ψ of the polynomial coefficients: [2489 7489 6478 6607]
INTT^ψ of the polynomial coefficients: [1 2 3 4]
INTT^ψ of the polynomial coefficients: [5 6 7 8]


>$\textbf{Proposition}$
>
>Let $\bm{a}$ and $\bm{b}$ are the multiplicands' vectors of polynomial coefficients.
>
>The negative-wrapped convolution of $\bm{a}$ and $\bm{b}$ can be calculated by:
>
>\begin{align*}
>    \bm{c} = \textsf{INTT}^{\psi^{-1}}(\textsf{NTT}^\psi(\bm{a}) \circ \textsf{NTT}^{\psi}(\bm{b}))
>\end{align*}
>
>where $\circ$ is an element-wise multiplication in $\Z_q$.

In [7]:
def NTT_like_NWC(poly1, poly2, psi, psi_inv, n, n_inv, q):
    ntt_poly1 = negative_wrapped_ntt(poly1, psi, q)
    ntt_poly2 = negative_wrapped_ntt(poly2, psi, q)

    ntt_res = np.zeros(n)
    for i in range(n):
        ntt_res[i] = (ntt_poly1[i] * ntt_poly2[i]) % q
    
    poly_nega_conv = negative_wrapped_intt(ntt_res, psi_inv, q, n_inv)
    return poly_nega_conv

# Polynomial coefficients a = [a_0, a_1, ..., a_{n-1}]
a = [1, 2, 3, 4]  # Example input polynomial
b = [5, 6, 7, 8]  # Example input polynomial
q = 7681  # Modulo value q
n = 4  # Degree n-1
n_inv = 5761
psi = 1925  # A primitive root or some suitable value for psi
psi_inv = 1213

res = NTT_like_NWC(a, b, psi, psi_inv, n, n_inv, q)
print(res)

[7625 7645    2   60]


---
# Simple Example

In [8]:
# Polynomial coefficients a = [a_0, a_1, ..., a_{n-1}]

a         = [1, 2, 3, 4]  # Example input polynomial
b         = [5, 6, 7, 8]  # Example input polynomial
q         = 7681          # Modulo value q
n         = 4             # Degree n-1
n_inv     = 5761
omega     = 3383          # A primitive root or some suitable value for psi
omega_inv = 4298
psi       = 1925          # A primitive root or some suitable value for psi
psi_inv   = 1213

res_p = NTT_like_PWC(a, b, omega, omega_inv, n, n_inv, q)
res_n = NTT_like_NWC(a, b, psi, psi_inv, n, n_inv, q)

print(res_p)
print(res_n)

[66 68 66 60]
[7625 7645    2   60]
